In [1]:
import requests as r
from lxml import html
from bs4 import BeautifulSoup as bs

import pandas as pd
import random
import time

Уже сделал часть работы с помощью `XPath` перед прошлым занятием, поэтому жалко было удалять, и я допилил этот вариант, чтобы по всем страницам проходил парсер и сделал 2й вариант с `BS4`

In [2]:
main_page = 'http://quotes.toscrape.com'
final_quotes_bs4 = []
final_quotes_xpath = []

### <font color="iceblue">Вариант с `BS4`</font>

In [3]:
def parse_res_bs4(body):
    '''Парсит все цитаты и возвращает список словарей'''
    qoutes_list = []
    quotes_elems = body.select('div[class=quote]')
    for q in quotes_elems:
        text = q.select_one('span[itemprop=text]').text
        author = q.select_one('span small[itemprop=author]').text
        tags = map(lambda t: t.text, q.find('div', class_='tags').find_all('a'))
        
        qoutes_list.append({
            'author': author,
            'text': text,
            'tags': list(tags)
        })
    
    return qoutes_list        

In [4]:
quotes_link = main_page

while True:
    time.sleep(random.randint(1, 5))
    print(f'Started scraping page {quotes_link}...')
    
    try:
        res = r.get(quotes_link, timeout=5)
        if res.status_code == 200:
            body = bs(res.content, 'html.parser')
            for i in parse_res_bs4(body):
                final_quotes_bs4.append(i)
                
            try:
                next_page_link = body.find('li', class_='next').select_one('a')['href']
                print(next_page_link)
                quotes_link = main_page + next_page_link
            except AttributeError as e:
                print('\n\nПарсер обработал последнюю страницу')
                break
            except Exception as e:
                print(f'Error: {e}')
                break
        else:
            print(f'Error: {res.status_code}')
            break
        
    except r.Timeout:
        print('Timeout')
        break

Started scraping page http://quotes.toscrape.com...
/page/2/
Started scraping page http://quotes.toscrape.com/page/2/...
/page/3/
Started scraping page http://quotes.toscrape.com/page/3/...
/page/4/
Started scraping page http://quotes.toscrape.com/page/4/...
/page/5/
Started scraping page http://quotes.toscrape.com/page/5/...
/page/6/
Started scraping page http://quotes.toscrape.com/page/6/...
/page/7/
Started scraping page http://quotes.toscrape.com/page/7/...
/page/8/
Started scraping page http://quotes.toscrape.com/page/8/...
/page/9/
Started scraping page http://quotes.toscrape.com/page/9/...
/page/10/
Started scraping page http://quotes.toscrape.com/page/10/...


Парсер обработал последнюю страницу


In [5]:
q_list_bs4 =  pd.DataFrame(final_quotes_bs4)
q_list_bs4

,author,text,tags
0,Albert Einstein,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,Albert Einstein,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"
...,...,...,...
95,Harper Lee,“You never really understand a person until yo...,[better-life-empathy]
96,Madeleine L'Engle,“You have to write the book that wants to be w...,"[books, children, difficult, grown-ups, write,..."
97,Mark Twain,“Never tell the truth to people who are not wo...,[truth]
98,Dr. Seuss,"“A person's a person, no matter how small.”",[inspirational]


### <font color="iceblue">Вариант с `XPath`</font>

In [6]:
def parse_result_xpath(body):
    qoutes_list = []
    qoutes = body.xpath("//div[contains(@class, 'quote')]")
    for q in qoutes:
        text = q.xpath(".//span[@itemprop='text']/text()")[0]
        author = q.xpath(".//span/small[@itemprop='author']/text()")[0]
        tags = q.xpath(".//div[@class='tags']/a[@class='tag']/text()")
        
        qoutes_list.append({
            'author': author,
            'text': text,
            'tags': tags
        })
    
    return qoutes_list

In [7]:
quotes_link = main_page

while True:
    time.sleep(random.randint(1, 5))
    print(f'Started scraping page {quotes_link}...')
    
    try:
        res = r.get(quotes_link, timeout=5)
        if res.status_code == 200:
            body = html.fromstring(res.content)
            for i in parse_result_xpath(body):
                final_quotes_xpath.append(i)
                
            try:
                next_page_link = body.xpath(".//nav/ul[contains(@class, 'pager')]/li[contains(@class, 'next')]/a/@href")[0]
                print(next_page_link)
                quotes_link = main_page + next_page_link
            except IndexError as e:
                print('\n\nПарсер обработал последнюю страницу')
                break
            except Exception as e:
                print(f'Error: {e}')
                break
        else:
            print(f'Error: {res.status_code}')
            break
        
    except r.Timeout:
        print('Timeout')
        break

Started scraping page http://quotes.toscrape.com...
/page/2/
Started scraping page http://quotes.toscrape.com/page/2/...
/page/3/
Started scraping page http://quotes.toscrape.com/page/3/...
/page/4/
Started scraping page http://quotes.toscrape.com/page/4/...
/page/5/
Started scraping page http://quotes.toscrape.com/page/5/...
/page/6/
Started scraping page http://quotes.toscrape.com/page/6/...
/page/7/
Started scraping page http://quotes.toscrape.com/page/7/...
/page/8/
Started scraping page http://quotes.toscrape.com/page/8/...
/page/9/
Started scraping page http://quotes.toscrape.com/page/9/...
/page/10/
Started scraping page http://quotes.toscrape.com/page/10/...


Парсер обработал последнюю страницу


In [8]:
q_list_xpath =  pd.DataFrame(final_quotes_xpath)
q_list_xpath

,author,text,tags
0,Albert Einstein,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,Albert Einstein,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"
...,...,...,...
95,Harper Lee,“You never really understand a person until yo...,[better-life-empathy]
96,Madeleine L'Engle,“You have to write the book that wants to be w...,"[books, children, difficult, grown-ups, write,..."
97,Mark Twain,“Never tell the truth to people who are not wo...,[truth]
98,Dr. Seuss,"“A person's a person, no matter how small.”",[inspirational]
